In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [3]:
survey = pd.read_csv("C:/Users/garci/Desktop/Data analytics/Analisis/Salud mental tecnologia/2014/survey.csv")
survey2016 = pd.read_csv("C:/Users/garci/Desktop/Data analytics/Analisis/Salud mental tecnologia/2016/mental-heath-in-tech-2016_20161114.csv")

In [4]:
survey.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


In [5]:
print(survey.keys())

Index(['Timestamp', 'Age', 'Gender', 'Country', 'state', 'self_employed',
       'family_history', 'treatment', 'work_interfere', 'no_employees',
       'remote_work', 'tech_company', 'benefits', 'care_options',
       'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'comments'],
      dtype='object')


In [6]:
len(survey)

1259

In [7]:
survey.isnull().sum()

Timestamp                       0
Age                             0
Gender                          0
Country                         0
state                         515
self_employed                  18
family_history                  0
treatment                       0
work_interfere                264
no_employees                    0
remote_work                     0
tech_company                    0
benefits                        0
care_options                    0
wellness_program                0
seek_help                       0
anonymity                       0
leave                           0
mental_health_consequence       0
phys_health_consequence         0
coworkers                       0
supervisor                      0
mental_health_interview         0
phys_health_interview           0
mental_vs_physical              0
obs_consequence                 0
comments                     1095
dtype: int64

In [8]:
#Limpiamos la columna de state, ya que queremos realizar el estudio a nivel global
survey = survey.drop(columns=['state'])

In [9]:
#Limpiamos los valores NA de la columna self_employed
survey = survey.dropna(subset=['self_employed'])

In [10]:
#Vemos cuantas veces aparece cada valor en work_interfere
survey['work_interfere'].value_counts(dropna=True)

work_interfere
Sometimes    458
Never        207
Rarely       170
Often        142
Name: count, dtype: int64

In [11]:
#Vemos el porcentaje de cada categoria
survey['work_interfere'].value_counts(normalize=True)


work_interfere
Sometimes    0.468782
Never        0.211873
Rarely       0.174002
Often        0.145343
Name: proportion, dtype: float64

In [12]:
survey = survey.dropna(subset=['work_interfere'])

In [13]:
#Comprobamos los datos de genero para limpiarlos
survey['Gender'].value_counts(dropna=True)

Gender
Male                                              471
male                                              156
Female                                            103
M                                                  85
female                                             51
F                                                  31
m                                                  23
f                                                  13
Make                                                4
Male                                                3
Cis Male                                            2
Female (trans)                                      2
Female                                              2
Cis Female                                          1
Male-ish                                            1
Trans-female                                        1
woman                                               1
Nah                                                 1
non-binary           

In [14]:
#pasamos todos los valores a minusculas y eliminamos cualquier espacion que puedan tener
survey['Gender'] = survey['Gender'].str.lower().str.strip()

In [15]:
survey['Gender'] = survey['Gender'].replace({
    'm' : 'male',
    'm' : 'male',
    'f' : 'female',
    'f' : 'female'
})

In [17]:
#Modificamos todos aquellos registros que no sean female o male por 'otros'
#survey['Gender'].mask(~survey['Gender'].isin(['female','male']),'others',inplace=True)
survey['Gender'] = survey['Gender'].mask(~survey['Gender'].isin(['female', 'male']), 'others')

In [18]:
##Nos guardamos todos los valores de edad fuera de rango para limpiar el dataset
survey['CleanedAge'] = survey['Age'].where((survey['Age'] > 16) & (survey['Age'] < 120))

In [19]:
survey = survey.drop(columns = 'Age')

In [20]:
#Limpiamos nulos
survey = survey.dropna(subset=['CleanedAge'])

In [24]:
survey['Age'] = survey.loc[:,'CleanedAge'].astype(int)

In [26]:
survey = survey.drop(columns = 'CleanedAge')

In [22]:
survey.to_excel("C:/Users/garci/Desktop/Data analytics/Analisis/Salud mental tecnologia/2014/cleanedDataset.xlsx")